In [1]:
import os
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\flash\__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [4]:
#download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="../data/imdb/train.csv",
    val_file="../data/imdb/valid.csv",
    test_file="../data/imdb/test.csv",
    batch_size=3
)

Map: 100%|██████████| 2500/2500 [00:00<00:00, 30122.41 examples/s]
c:\Users\Roma\.virtualenvs\Practical-Deep-Learning-at-Scale-with-MLFl-NGqyZvc3\lib\site-packages\IPython\core\interactiveshell.py:3550: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:

EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://localhost')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

2023/11/24 12:24:51 INFO mlflow.tracking.fluent: Experiment with name 'dl_model_chapter03' does not exist. Creating a new experiment.


experiment_id: 1


In [7]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test(datamodule=datamodule)

2023/11/24 12:25:16 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
Missing logger folder: c:\Users\Roma\Documents\Programming\Practical-Deep-Learning-at-Scale-with-MLFlow\chapter03\lightning_logs

  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Epoch 2: 100%|██████████| 8334/8334 [02:54<00:00, 47.80it/s, loss=0.707, v_num=0, train_accuracy_step=1.000, train_cross_entropy_step=0.370, val_accuracy=0.652, val_cross_entropy=0.632, train_accuracy_epoch=0.615, train_cross_entropy_epoch=0.655]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 8334/8334 [02:54<00:00, 47.77it/s, loss=0.707, v_num=0, train_accuracy_step=1.000, train_cross_entropy_step=0.370, val_accuracy=0.652, val_cross_entropy=0.632, train_accuracy_epoch=0.615, train_cross_entropy_epoch=0.655]


INFO: Restoring states from the checkpoint path at c:\Users\Roma\Documents\Programming\Practical-Deep-Learning-at-Scale-with-MLFlow\chapter03\lightning_logs\version_0\checkpoints\epoch=2-step=22500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at c:\Users\Roma\Documents\Programming\Practical-Deep-Learning-at-Scale-with-MLFlow\chapter03\lightning_logs\version_0\checkpoints\epoch=2-step=22500.ckpt
INFO: Loaded model weights from checkpoint at c:\Users\Roma\Documents\Programming\Practical-Deep-Learning-at-Scale-with-MLFlow\chapter03\lightning_logs\version_0\checkpoints\epoch=2-step=22500.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from checkpoint at c:\Users\Roma\Documents\Programming\Practical-Deep-Learning-at-Scale-with-MLFlow\chapter03\lightning_logs\version_0\checkpoints\epoch=2-step=22500.ckpt


Testing DataLoader 0: 100%|██████████| 834/834 [00:13<00:00, 64.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6528000235557556
   test_cross_entropy       0.6299830079078674
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [8]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: 29bffeb1f278405a86b8a16d41e8ddd5; lifecycle_stage: active


In [13]:
# use the run_id to construct a logged_model URI. An example is shown here:
# logged_model = 'runs:/37a3fe9b6faf41d89001eca13ad6ca47/model'
logged_model = f'runs:/{run_id}/model'


# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
model.predict({'What a piece of disappointing news'})


AttributeError: `flash.Task.predict` has been removed. Use `flash.Trainer.predict` instead.

In [15]:
# register the model
model_registry_version = mlflow.register_model(logged_model, 'nlp_dl_model')
print(f'Model Name: {model_registry_version.name}')
print(f'Model Version: {model_registry_version.version}')

Successfully registered model 'nlp_dl_model'.
2023/11/24 12:37:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nlp_dl_model, version 1


Model Name: nlp_dl_model
Model Version: 1


Created version '1' of model 'nlp_dl_model'.
